In [1]:
import json
import kfp
import kfp.dsl as dsl
import kfp.components as comp
from kfp.components import OutputPath

#### Pipeline Configurations

In [2]:
#The Google Cloud project that this pipeline runs in.
project_id = "jads-de-2021"
# The region that this pipeline runs in
region = "us-west1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
pipeline_root_path = "gs://dejads_temp_yk"

#### Create the Components from Component Specifications 

In [7]:
from kfp.components import load_component_from_file

data_ingestion_op = load_component_from_file(
    './../components/data_ingestor_component.yaml')

mlp_model_op = load_component_from_file(
    './../components/mlp_trainer_component.yaml')

#### Define the Pipeline

In [8]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="lab9-v2",
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str):
    
    # The first step    
    di_op = data_ingestion_op(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )

    # The second step 
    training_job_run_op = mlp_model_op(
        project_id=project_id,
        model_repo=model_repo,       
        feature_path=di_op.outputs['output_path']
    )

#### Compile the Pipeline

In [9]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='diabetes_predictor_pipeline_v2.json')

#### Run the Pipeline

In [10]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="lab9-v1-diabetes-predictor-v2",
    template_path="diabetes_predictor_pipeline_v2.json",
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id,
        'data_bucket': 'dejads_input_yk',
        'trainset_filename': 'training_set.csv',
        'model_repo':'models-de-lab9'
    }
)

job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/411957191814/locations/us-central1/pipelineJobs/lab9-v2-20211122125305
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/411957191814/locations/us-central1/pipelineJobs/lab9-v2-20211122125305')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/lab9-v2-20211122125305?project=411957191814
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob run completed. Resource name: projects/411957191814/locations/us-central1/pipelineJobs/lab9-v2-20211122125305
